# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
dir = '/content/drive/Shared drives/bds-final-project'

Mounted at /content/drive


## Reading in the data

In [ ]:
col_names = ['id', 'device_id', 'vehicle_type', 'trip_duration', 'trip_distance', 'start_time', 'end_time', 'modified_date', 'month', 'hour', 'day_of_week', 'district_start', 'district_end', 'year', 'tract_start', 'tract_end']
df = pd.read_csv(f'{dir}/raw-data/Shared_Micromobility_Vehicle_Trips.csv', names=col_names, skiprows=[0])
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,device_id,vehicle_type,trip_duration,trip_distance,start_time,end_time,modified_date,month,hour,day_of_week,district_start,district_end,year,tract_start,tract_end
0,1d4f113d-71b2-4213-9170-1644e048d4a1,d4d91f54-5527-4bef-9ce8-24808fbfadc3,scooter,420,869.0,04/10/2018 08:00:00 PM,04/10/2018 08:00:00 PM,04/17/2019 01:58:49 AM,4,20,2,9,9,2018,48453001100,48453001100
1,05d3d08f-9960-48e3-82f6-79a375ee28b0,4f62c903-e21b-499f-9197-fd0498f25cc5,scooter,371,1037.0,04/10/2018 07:45:00 PM,04/10/2018 08:00:00 PM,04/17/2019 01:58:49 AM,4,19,2,9,9,2018,48453000603,48453000601
2,61a4992e-7dff-4142-8dd6-baa23acf561a,6a25bdcf-1f68-4778-95b4-4223c7beec15,scooter,1687,4491.0,04/23/2018 01:15:00 PM,04/23/2018 01:45:00 PM,04/17/2019 02:42:15 AM,4,13,1,9,9,2018,48453001100,48453001100
3,f1410b76-327b-479c-825e-45168034f390,b6efa85b-a836-4412-9620-77609f0e612b,scooter,524,633.0,04/27/2018 09:45:00 PM,04/27/2018 10:00:00 PM,04/17/2019 02:42:15 AM,4,21,5,9,9,2018,48453000603,48453000604
4,61250373-bece-46c2-a7c6-fb91c53eeefe,7af82887-fe34-4327-a63a-dfd11646252f,scooter,322,1114.0,04/27/2018 09:45:00 PM,04/27/2018 09:45:00 PM,04/17/2019 02:42:15 AM,4,21,5,9,9,2018,48453000500,48453000601


## Dropping Nulls and Unnessecary Columns

In [ ]:
df_clean = df.copy()

In [ ]:
print(df_clean.shape)

(9959867, 16)


In [ ]:
# Checking for null values
df_clean.isnull().sum()

id                  0
device_id           0
vehicle_type        0
trip_duration       0
trip_distance       0
start_time          0
end_time            0
modified_date       0
month               0
hour                0
day_of_week         0
district_start    134
district_end      134
year                0
tract_start       135
tract_end         135
dtype: int64

In [ ]:
# Replacing empty cells with np.NaN
cols = ['district_start', 'district_end', 'tract_start', 'tract_end']
df_clean[cols] = df_clean[cols].replace(['None','OUT_OF_BOUNDS'], np.nan)

In [ ]:
# Checking for null values after replace
df_clean.isnull().sum()

id                    0
device_id             0
vehicle_type          0
trip_duration         0
trip_distance         0
start_time            0
end_time              0
modified_date         0
month                 0
hour                  0
day_of_week           0
district_start      792
district_end       1171
year                  0
tract_start       41496
tract_end         31723
dtype: int64

In [ ]:
# Dropping null values
df_clean = df_clean.dropna(axis=0, how='any')
df_clean.isnull().sum()

id                0
device_id         0
vehicle_type      0
trip_duration     0
trip_distance     0
start_time        0
end_time          0
modified_date     0
month             0
hour              0
day_of_week       0
district_start    0
district_end      0
year              0
tract_start       0
tract_end         0
dtype: int64

In [ ]:
# Applying numeric type to cleaned columns
df_clean[cols] = df_clean[cols].apply(pd.to_numeric)

In [ ]:
# Dropping some unnessecary columns
df_clean.drop(['id', 'device_id', 'modified_date'], axis=1, inplace=True)

In [ ]:
# Checking dtypes of columns after cleaning null values
df_clean.dtypes

vehicle_type       object
trip_duration       int64
trip_distance     float64
start_time         object
end_time           object
month               int64
hour                int64
day_of_week         int64
district_start    float64
district_end      float64
year                int64
tract_start       float64
tract_end         float64
dtype: object

## Converting datetimes to unix timestamps

In [ ]:
df_clean.head()

,vehicle_type,trip_duration,trip_distance,start_time,end_time,month,hour,day_of_week,district_start,district_end,year,tract_start,tract_end
0,scooter,420,869.0,04/10/2018 08:00:00 PM,04/10/2018 08:00:00 PM,4,20,2,9.0,9.0,2018,4.845300e+10,4.845300e+10
1,scooter,371,1037.0,04/10/2018 07:45:00 PM,04/10/2018 08:00:00 PM,4,19,2,9.0,9.0,2018,4.845300e+10,4.845300e+10
2,scooter,1687,4491.0,04/23/2018 01:15:00 PM,04/23/2018 01:45:00 PM,4,13,1,9.0,9.0,2018,4.845300e+10,4.845300e+10
3,scooter,524,633.0,04/27/2018 09:45:00 PM,04/27/2018 10:00:00 PM,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10
4,scooter,322,1114.0,04/27/2018 09:45:00 PM,04/27/2018 09:45:00 PM,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10


In [ ]:
time_cols = ['start_time', 'end_time']
df_clean[time_cols] = df_clean[time_cols].apply(pd.to_datetime)

In [ ]:
df_clean[time_cols] = df_clean[time_cols].values.astype(np.int64) // pd.Timedelta(1, 's').value

In [ ]:
df_clean.head()

,vehicle_type,trip_duration,trip_distance,start_time,end_time,month,hour,day_of_week,district_start,district_end,year,tract_start,tract_end
0,scooter,420,869.0,1523390400,1523390400,4,20,2,9.0,9.0,2018,4.845300e+10,4.845300e+10
1,scooter,371,1037.0,1523389500,1523390400,4,19,2,9.0,9.0,2018,4.845300e+10,4.845300e+10
2,scooter,1687,4491.0,1524489300,1524491100,4,13,1,9.0,9.0,2018,4.845300e+10,4.845300e+10
3,scooter,524,633.0,1524865500,1524866400,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10
4,scooter,322,1114.0,1524865500,1524865500,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10


## One-hot encoding `vehicle_type`

In [ ]:
print(df_clean.vehicle_type.nunique())
df_clean.vehicle_type.value_counts()

3


scooter    9363696
bicycle     451151
moped        87413
Name: vehicle_type, dtype: int64

In [ ]:
dummies = pd.get_dummies(df_clean.vehicle_type, prefix='vehicle_type')
df_clean = df_clean.drop('vehicle_type', axis=1)
df_clean = df_clean.join(dummies)
df_clean.head() 

,trip_duration,trip_distance,start_time,end_time,month,hour,day_of_week,district_start,district_end,year,tract_start,tract_end,vehicle_type_bicycle,vehicle_type_moped,vehicle_type_scooter
0,420,869.0,1523390400,1523390400,4,20,2,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
1,371,1037.0,1523389500,1523390400,4,19,2,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
2,1687,4491.0,1524489300,1524491100,4,13,1,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
3,524,633.0,1524865500,1524866400,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
4,322,1114.0,1524865500,1524865500,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1


In [ ]:
df_clean.shape

(9902260, 15)

In [ ]:
df_clean.to_csv(f'{dir}/scooter_data_clean.csv')

In [ ]:
# Just making sure the clean .csv didn't get messed up
df_test = pd.read_csv(f'{dir}/scooter_data_clean.csv', index_col=0)
df_test.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,trip_duration,trip_distance,start_time,end_time,month,hour,day_of_week,district_start,district_end,year,tract_start,tract_end,vehicle_type_bicycle,vehicle_type_moped,vehicle_type_scooter
0,420,869.0,1523390400,1523390400,4,20,2,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
1,371,1037.0,1523389500,1523390400,4,19,2,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
2,1687,4491.0,1524489300,1524491100,4,13,1,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
3,524,633.0,1524865500,1524866400,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
4,322,1114.0,1524865500,1524865500,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1


# Grouping by day

In [ ]:
df_grp = pd.read_csv(f'{dir}/scooter_data_clean.csv', index_col=0)
df_grp.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,trip_duration,trip_distance,start_time,end_time,month,hour,day_of_week,district_start,district_end,year,tract_start,tract_end,vehicle_type_bicycle,vehicle_type_moped,vehicle_type_scooter
0,420,869.0,1523390400,1523390400,4,20,2,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
1,371,1037.0,1523389500,1523390400,4,19,2,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
2,1687,4491.0,1524489300,1524491100,4,13,1,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
3,524,633.0,1524865500,1524866400,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1
4,322,1114.0,1524865500,1524865500,4,21,5,9.0,9.0,2018,4.845300e+10,4.845300e+10,0,0,1


In [ ]:
df_grp['date'] = pd.to_datetime(df_grp.start_time, unit='s')

In [ ]:
df_grp = df_grp.groupby([df_grp['date'].dt.date, df_grp['day_of_week']]).agg({
    'trip_duration': 'mean',
    'trip_distance': 'mean',
    'vehicle_type_bicycle': 'sum',
    'vehicle_type_moped': 'sum',
    'vehicle_type_scooter': 'sum',
})

In [ ]:
df_grp = df_grp.rename({
    'trip_duration': 'avg_trip_duration',
    'trip_distance': 'avg_trip_distance',
    'vehicle_type_bicycle': 'num_bicycle_trips',
    'vehicle_type_moped': 'num_moped_trips',
    'vehicle_type_scooter': 'num_scooter_trips'
}, axis=1)

In [ ]:
df_grp.head()

,,avg_trip_duration,avg_trip_distance,num_bicycle_trips,num_moped_trips,num_scooter_trips
date,day_of_week,,,,,
2018-04-03,2,943.000000,419.000000,0,0,1
2018-04-04,3,1360.333333,5691.333333,0,0,3
2018-04-05,4,1198.633929,2238.776786,0,0,112
2018-04-06,5,1118.914530,2616.518519,0,0,351
2018-04-07,6,970.707207,1815.882883,0,0,222


In [ ]:
df_grp = df_grp.reset_index()

In [ ]:
df_grp.head()

,date,day_of_week,avg_trip_duration,avg_trip_distance,num_bicycle_trips,num_moped_trips,num_scooter_trips
0,2018-04-03,2,943.000000,419.000000,0,0,1
1,2018-04-04,3,1360.333333,5691.333333,0,0,3
2,2018-04-05,4,1198.633929,2238.776786,0,0,112
3,2018-04-06,5,1118.914530,2616.518519,0,0,351
4,2018-04-07,6,970.707207,1815.882883,0,0,222


In [ ]:
df_grp.to_csv(f'{dir}/mobility_data_by_day.csv')

## Weather Data

In [ ]:
df_weather = pd.read_csv(f'{dir}/raw-data/Austin_WeatherData_2018-20.csv')
df_weather.head()

,Date,Max_Temperature_F,Avg_Temperature_F,Min_Temperature_F,Max_DewPoint_F,Avg_DewPoint_F,Min_DewPoint_F,Max_Humidity_%,Avg_Humidity_%,Min_Humidity_%,Max_WindSpeed_mph,Avg_WindSpeed_mph,Min_WindSpeed_mph,Max_Pressure_Hg,Avg_Pressure_Hg,Min_Pressure_Hg,Precipitation_inches
0,4/1/2018,82,71.2,64,66,63.1,61,93,76.8,54,13,9.3,5,29.4,29.4,29.3,0.0
1,4/2/2018,85,70.8,58,67,62.7,54,90,76.7,51,20,9.7,0,29.3,29.3,29.1,0.0
2,4/3/2018,80,72.7,65,71,64.4,40,90,76.6,40,24,12.1,0,29.5,29.2,29.1,0.0
3,4/4/2018,70,59.8,50,49,35.1,30,80,41.6,26,28,13.5,0,29.7,29.6,29.5,0.0
4,4/5/2018,78,68.0,56,67,57.6,47,87,70.0,56,16,9.0,0,29.6,29.4,29.3,0.0


In [ ]:
df_weather.describe()

,Max_Temperature_F,Avg_Temperature_F,Min_Temperature_F,Max_DewPoint_F,Avg_DewPoint_F,Min_DewPoint_F,Max_Humidity_%,Avg_Humidity_%,Min_Humidity_%,Max_WindSpeed_mph,Avg_WindSpeed_mph,Min_WindSpeed_mph,Max_Pressure_Hg,Avg_Pressure_Hg,Min_Pressure_Hg,Precipitation_inches
count,963.000000,963.000000,963.000000,963.000000,963.000000,963.000000,963.000000,963.000000,963.000000,963.000000,963.000000,963.000000,963.000000,963.000000,963.000000,963.000000
mean,82.448598,70.577466,60.007269,64.167186,59.105192,52.944964,92.158879,70.739979,44.524403,16.451713,8.101246,1.605400,29.506957,29.427103,29.286189,0.092866
std,14.433308,13.622001,14.988594,12.987043,14.019915,15.351261,6.624993,11.509974,16.261640,5.219955,3.523839,2.810371,0.160377,0.172913,1.345952,0.353963
min,39.000000,34.000000,0.000000,23.000000,16.300000,0.000000,40.000000,31.600000,0.000000,6.000000,1.100000,0.000000,29.100000,27.200000,0.000000,0.000000
25%,73.000000,59.800000,48.000000,57.000000,49.250000,40.000000,90.000000,63.800000,32.000000,13.000000,5.300000,0.000000,29.400000,29.300000,29.300000,0.000000
50%,85.000000,73.400000,64.000000,69.000000,64.100000,58.000000,93.000000,70.600000,42.000000,16.000000,7.700000,0.000000,29.500000,29.400000,29.300000,0.000000
75%,95.000000,82.550000,73.000000,74.000000,70.750000,66.000000,97.000000,79.200000,54.000000,20.000000,10.550000,3.000000,29.600000,29.500000,29.400000,0.000000
max,109.000000,93.400000,84.000000,79.000000,77.200000,76.000000,100.000000,94.600000,89.000000,41.000000,19.800000,14.000000,30.100000,30.000000,29.900000,3.950000


In [ ]:
df_weather.shape

(963, 17)

In [ ]:
df_weather.isna().sum()

Date                    0
Max_Temperature_F       0
Avg_Temperature_F       0
Min_Temperature_F       0
Max_DewPoint_F          0
Avg_DewPoint_F          0
Min_DewPoint_F          0
Max_Humidity_%          0
Avg_Humidity_%          0
Min_Humidity_%          0
Max_WindSpeed_mph       0
Avg_WindSpeed_mph       0
Min_WindSpeed_mph       0
Max_Pressure_Hg         0
Avg_Pressure_Hg         0
Min_Pressure_Hg         0
Precipitation_inches    0
dtype: int64

In [ ]:
df_weather.dtypes

Date                     object
Max_Temperature_F         int64
Avg_Temperature_F       float64
Min_Temperature_F         int64
Max_DewPoint_F            int64
Avg_DewPoint_F          float64
Min_DewPoint_F            int64
Max_Humidity_%            int64
Avg_Humidity_%          float64
Min_Humidity_%            int64
Max_WindSpeed_mph         int64
Avg_WindSpeed_mph       float64
Min_WindSpeed_mph         int64
Max_Pressure_Hg         float64
Avg_Pressure_Hg         float64
Min_Pressure_Hg         float64
Precipitation_inches    float64
dtype: object

In [ ]:
df_mobility = pd.read_csv(f'{dir}/mobility_data_by_day.csv', index_col=0)
print(df_mobility.shape)
df_mobility.head()

(1452, 7)


,date,day_of_week,avg_trip_duration,avg_trip_distance,num_bicycle_trips,num_moped_trips,num_scooter_trips
0,2018-04-03,2,943.000000,419.000000,0,0,1
1,2018-04-04,3,1360.333333,5691.333333,0,0,3
2,2018-04-05,4,1198.633929,2238.776786,0,0,112
3,2018-04-06,5,1118.914530,2616.518519,0,0,351
4,2018-04-07,6,970.707207,1815.882883,0,0,222


In [ ]:
# Convert both date columns to datetimes for easy comparison
df_weather.Date = df_weather.Date.apply(pd.to_datetime)
df_mobility.date = df_mobility.date.apply(pd.to_datetime)

In [ ]:
df_weather = df_weather.rename({'Date': 'date'}, axis=1)
merged = pd.merge(df_mobility, df_weather, how='inner', on='date')

In [ ]:
merged

,date,day_of_week,avg_trip_duration,avg_trip_distance,num_bicycle_trips,num_moped_trips,num_scooter_trips,Max_Temperature_F,Avg_Temperature_F,Min_Temperature_F,Max_DewPoint_F,Avg_DewPoint_F,Min_DewPoint_F,Max_Humidity_%,Avg_Humidity_%,Min_Humidity_%,Max_WindSpeed_mph,Avg_WindSpeed_mph,Min_WindSpeed_mph,Max_Pressure_Hg,Avg_Pressure_Hg,Min_Pressure_Hg,Precipitation_inches
0,2018-04-03,2,943.000000,419.000000,0,0,1,80,72.7,65,71,64.4,40,90,76.6,40,24,12.1,0,29.5,29.2,29.1,0.00
1,2018-04-04,3,1360.333333,5691.333333,0,0,3,70,59.8,50,49,35.1,30,80,41.6,26,28,13.5,0,29.7,29.6,29.5,0.00
2,2018-04-05,4,1198.633929,2238.776786,0,0,112,78,68.0,56,67,57.6,47,87,70.0,56,16,9.0,0,29.6,29.4,29.3,0.00
3,2018-04-06,5,1118.914530,2616.518519,0,0,351,85,75.6,71,71,68.7,67,93,80.1,58,22,12.4,3,29.3,29.2,29.1,0.00
4,2018-04-07,6,970.707207,1815.882883,0,0,222,72,50.0,41,69,44.6,35,96,81.6,73,28,18.4,12,29.5,29.4,29.2,0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,2020-11-14,5,844.764112,1831.127309,438,213,10935,88,76.2,66,69,66.0,61,91,73.2,42,18,9.5,0,29.4,29.3,29.2,0.00
1446,2020-11-14,6,3053.381410,295615.108854,8,31,273,88,76.2,66,69,66.0,61,91,73.2,42,18,9.5,0,29.4,29.3,29.2,0.00
1447,2020-11-15,0,2980.400000,4465.690181,2,9,139,73,62.4,39,69,34.4,20,93,44.0,14,28,12.2,0,29.7,29.6,29.4,0.00
1448,2020-11-15,6,865.656728,10947.714433,396,239,11909,73,62.4,39,69,34.4,20,93,44.0,14,28,12.2,0,29.7,29.6,29.4,0.00


In [ ]:
merged.to_csv(f'{dir}/merged_data.csv')

In [4]:
df = pd.read_csv(f'{dir}/merged_data.csv', index_col=0)
df.head()

,date,day_of_week,avg_trip_duration,avg_trip_distance,num_bicycle_trips,num_moped_trips,num_scooter_trips,Max_Temperature_F,Avg_Temperature_F,Min_Temperature_F,Max_DewPoint_F,Avg_DewPoint_F,Min_DewPoint_F,Max_Humidity_%,Avg_Humidity_%,Min_Humidity_%,Max_WindSpeed_mph,Avg_WindSpeed_mph,Min_WindSpeed_mph,Max_Pressure_Hg,Avg_Pressure_Hg,Min_Pressure_Hg,Precipitation_inches
0,2018-04-03,2,943.000000,419.000000,0,0,1,80,72.7,65,71,64.4,40,90,76.6,40,24,12.1,0,29.5,29.2,29.1,0.00
1,2018-04-04,3,1360.333333,5691.333333,0,0,3,70,59.8,50,49,35.1,30,80,41.6,26,28,13.5,0,29.7,29.6,29.5,0.00
2,2018-04-05,4,1198.633929,2238.776786,0,0,112,78,68.0,56,67,57.6,47,87,70.0,56,16,9.0,0,29.6,29.4,29.3,0.00
3,2018-04-06,5,1118.914530,2616.518519,0,0,351,85,75.6,71,71,68.7,67,93,80.1,58,22,12.4,3,29.3,29.2,29.1,0.00
4,2018-04-07,6,970.707207,1815.882883,0,0,222,72,50.0,41,69,44.6,35,96,81.6,73,28,18.4,12,29.5,29.4,29.2,0.31


In [7]:
df.date.min(), df.date.max()

('2018-04-03', '2020-11-16')

In [8]:
df.shape

(1450, 23)